# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "../WeatherPy/cities.csv"

# Read cities file and store into Pandas data frame
cities_df = pd.read_csv(file_to_load)

#Clean column
cleaned_cities_df = cities_df.drop(["Unnamed: 0"], axis=1)

#Display dataframe
cleaned_cities_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nanortalik,99,GL,1609795197,96,60.14,-45.24,27.12,8.41
1,Padang,90,ID,1609795199,88,-0.95,100.35,73.40,1.12
2,Carnarvon,75,AU,1609795200,94,-24.87,113.63,71.60,6.93
3,Vaini,75,TO,1609795203,88,-21.20,-175.20,80.60,9.17
4,Tehachapi,1,US,1609794962,44,35.13,-118.45,53.60,18.34
...,...,...,...,...,...,...,...,...,...
561,Ossora,73,RU,1609795906,83,59.24,163.07,-8.48,7.83
562,Ţaga,90,RO,1609795908,100,46.95,24.05,41.00,4.63
563,Dunedin,100,NZ,1609795693,88,-45.87,170.50,55.20,9.01
564,Penzance,78,GB,1609795911,77,50.12,-5.54,39.00,20.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [31]:
# Store 'Lat' and 'Lng' into  locations
locations = cleaned_cities_df[["Lat", "Lng"]].astype(float)

#Store humidity
humidity = cities_df['Humidity'].astype(float)

# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Included a .copy to get rid of "setting with copy" warning
best_weather_cities_df = cleaned_cities_df.loc[(cleaned_cities_df["Max Temp"]>70) & (cleaned_cities_df["Max Temp"]<90) & (cleaned_cities_df['Wind Speed']<10)
                                      & (cleaned_cities_df['Cloudiness'] == 0)].copy()

#Check count for cities
best_weather_cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,Lüderitz,0,NaN,1609795270,73,-26.65,15.16,71.60,3.67
76,Charlestown,0,AU,1609795300,97,-32.95,151.67,71.01,3.27
90,Arlit,0,NE,1609795317,17,18.74,7.39,78.66,6.17
119,Acaponeta,0,MX,1609795357,20,22.50,-105.36,89.01,5.91
169,Ahipara,0,NZ,1609795419,57,-35.17,173.17,71.96,0.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Add column for Hotel Names
best_weather_cities_df['Hotel Name'] = ""

#Check DF
best_weather_cities_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,Lüderitz,0,NaN,1609795270,73,-26.65,15.16,71.60,3.67,
76,Charlestown,0,AU,1609795300,97,-32.95,151.67,71.01,3.27,
90,Arlit,0,NE,1609795317,17,18.74,7.39,78.66,6.17,
119,Acaponeta,0,MX,1609795357,20,22.50,-105.36,89.01,5.91,
169,Ahipara,0,NZ,1609795419,57,-35.17,173.17,71.96,0.67,


In [29]:
#Iterate over rows
hotels = []

for i in range(len(best_weather_cities_df)):
    lat = best_weather_cities_df.iloc[i]['Lat']
    lng = best_weather_cities_df.iloc[i]['Lng']
    

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "keyword": "hotel",
        "key": g_key
    }

    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotels.append(response['results'][i]['name'])
        
    except:
        hotels.append("None")

In [30]:
#Add hotels data to df
best_weather_cities_df["Hotel Name"] = hotels
best_weather_cities_df.head(24)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,Lüderitz,0,NaN,1609795270,73,-26.65,15.16,71.60,3.67,LÜDERITZ NEST HOTEL
76,Charlestown,0,AU,1609795300,97,-32.95,151.67,71.01,3.27,Mercure Charlestown
90,Arlit,0,NE,1609795317,17,18.74,7.39,78.66,6.17,None
119,Acaponeta,0,MX,1609795357,20,22.50,-105.36,89.01,5.91,Hotel San Luis
169,Ahipara,0,NZ,1609795419,57,-35.17,173.17,71.96,0.67,Beach Abode Beachfront Lodge
176,Korhogo,0,CI,1609795428,21,9.45,-5.63,81.57,2.73,Hotel Bon Sejour
197,Ngunguru,0,NZ,1609795456,70,-35.62,174.50,73.99,1.99,None
223,San Carlos,0,VE,1609795488,34,9.66,-68.58,86.74,2.84,Residencia Don Pepe
239,Am Timan,0,TD,1609795508,19,11.03,20.28,75.45,4.47,None
282,Kumo,0,NG,1609795565,20,10.04,11.21,77.41,6.55,None


In [38]:
#Drop None Values
hotel_df = best_weather_cities_df.loc[(best_weather_cities_df["Hotel Name"] != 'None')]

#Show new dataframe with hotels within 5000 meteres only 
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,Lüderitz,0,NaN,1609795270,73,-26.65,15.16,71.60,3.67,LÜDERITZ NEST HOTEL
76,Charlestown,0,AU,1609795300,97,-32.95,151.67,71.01,3.27,Mercure Charlestown
119,Acaponeta,0,MX,1609795357,20,22.50,-105.36,89.01,5.91,Hotel San Luis
169,Ahipara,0,NZ,1609795419,57,-35.17,173.17,71.96,0.67,Beach Abode Beachfront Lodge
176,Korhogo,0,CI,1609795428,21,9.45,-5.63,81.57,2.73,Hotel Bon Sejour
223,San Carlos,0,VE,1609795488,34,9.66,-68.58,86.74,2.84,Residencia Don Pepe


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))